(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Analyzing Hotel Ratings on Tripadvisor

In this homework we will focus on practicing two techniques: web scraping and regression. For the first part, we will get some basic information for each hotel in Boston. Then, we will fit a regression model on this information and try to analyze it.

** Task 1 (30 pts)**

We will scrape the data using Beautiful Soup. For each hotel that our search returns, we will get the information below.

![Information to be scraped](hotel_info.png)

Of course, feel free to collect even more data if you want. 

In [148]:
import urllib2
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
url_list = []
page = urllib2.urlopen('https://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html').read()
soup = BeautifulSoup(page, "lxml")
list = soup.findAll('div', attrs = {'class':'prw_rup prw_common_short_cell_thumbnail'})
myNext = soup.findAll('a', attrs = {'class':'nav next ui_button primary taLnk'})
for s in list:
    tmp = BeautifulSoup(str(s),"lxml")
    tmpList = tmp.findAll('a', href=True)
    for element in tmpList:
        arr = str(element).split(' ')
        tmp_arr = str(arr[1]).split('=')
        url = str(tmp_arr[1])[2:-1]
        url_list.append(url)
        
while len(myNext) == 1:
    nextTmp = myNext[0]
    st = nextTmp.get('href')
    url = 'https://www.tripadvisor.com' + st
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    list = soup.findAll('div', attrs = {'class':'prw_rup prw_common_short_cell_thumbnail'})
    myNext = soup.findAll('a', attrs = {'class':'nav next ui_button primary taLnk'})
    for s in list:
        tmp = BeautifulSoup(str(s),"lxml")
        tmpList = tmp.findAll('a', href=True)
        for element in tmpList:
            arr = str(element).split(' ')
            tmp_arr = str(arr[1]).split('=')
            url = str(tmp_arr[1])[2:-1]
            url_list.append(url)
        
url_list = set(url_list)
target_rating = []
count = 0;
review_url = []
for url in url_list:
    url = 'https://www.tripadvisor.com/' + url
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    innerBubble = soup.findAll('div', attrs = {'class':'reviewSelector'})
    tmp =  BeautifulSoup(str(innerBubble[0]), "lxml")
    tmp_page = tmp.findAll('div', attrs = {'class':'quote'})
    div = BeautifulSoup(str(tmp_page), "lxml")
    div_html = BeautifulSoup(str(div), "lxml")
    buff = div_html.findAll('a', href = True)
    #buff[0]['href']
    if(len(buff) == 1):
        review_url.append(buff[0]['href']);
        
for url in review_url:
    url = 'https://www.tripadvisor.com' + url
    print url
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    element = soup.findAll('div', attrs = {'class':'col rating '})
    element = BeautifulSoup(str(element), "lxml")
    temp = element.findAll('li')
    temp = BeautifulSoup(str(temp), "lxml")
    span = temp.findAll('span', attrs={'class': None})
    test = str(span).split('<span>')
    
    num1 = int(test[1].replace(",", ""))
    num2 = int(test[4].replace(",", ""))
    num3 = int(test[7].replace(",", ""))
    num4 = int(test[10].replace(",", ""))
    num5 = int(test[13].replace(",", ""))
    
    y = num1 * 5 + num2 + 4 + num3 * 3 + num4 * 2 + num5 * 1
    y = float(y) / (num1 + num2 + num3 + num4 + num5)
    y = round(y, 2)
    target_rating.append(y)

df = pd.DataFrame(review_url)
df.to_csv('url.csv')

#print target_rating

https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#CHECK_RATES_CONT
https://www.tripadvisor.com/ShowUserReviews-g60745-d217550-r356816590-DoubleTree_by_Hilton_Hotel_Boston_Downtown-Boston_Massachusetts.html#CHECK_RATES_CONT
https://www.tripadvisor.com/ShowUserReviews-g60745-d220216-r349646225-Beacon_Hill_Hotel_and_Bistro-Boston_Massachusetts.html#CHECK_RATES_CONT
https://www.tripadvisor.com/ShowUserReviews-g60745-d89568-r356817722-Chandler_Inn-Boston_Massachusetts.html#CHECK_RATES_CONT
https://www.tripadvisor.com/ShowUserReviews-g60745-d225873-r352387745-Charlesmark_Hotel-Boston_Massachusetts.html#CHECK_RATES_CONT
https://www.tripadvisor.com/ShowUserReviews-g60745-d222958-r355365182-Comfort_Inn_Boston-Boston_Massachusetts.html#CHECK_RATES_CONT
https://www.tripadvisor.com/ShowUserReviews-g60745-d111441-r356895121-Hyatt_Regency_Boston-Boston_Massachusetts.html#CHECK_RATES_CONT
https://www.tripadvisor.com/ShowUserReviews-g60745-d22295

'\nfor url in review_url:\n    url = \'https://www.tripadvisor.com\' + url\n    page = urllib2.urlopen(url).read()\n    soup = BeautifulSoup(page, "lxml")\n    innerBubble = soup.findAll(\'div\', attrs = {\'class\':\'reviewSelector\'})\n    tmp =  BeautifulSoup(str(innerBubble[0]), "lxml")\n'

In [14]:
import urllib2
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = []
with open('url.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        arr = str(row).split('/')
        url.append(arr[1][:-2])
        
driver = webdriver.Firefox()
for element in url:
    value = 0
    location = 0
    s_quality = 0
    rooms = 0
    clean = 0
    service = 0
    total = 0
    c = 0
    link = 'https://www.tripadvisor.com/' + element
    page = urllib2.urlopen(link).read()
    soup = BeautifulSoup(page, "lxml")
    current = soup.findAll('span', attrs = {'class':'pageNum current'})
    my_next = soup.findAll('a', attrs = {'class':'nav next rndBtn ui_button primary taLnk'})
    innerBubble = soup.findAll('div', attrs = {'class':'innerBubble'})
    buff = BeautifulSoup(str(innerBubble), "lxml")
    recommend = buff.findAll('ul', attrs = {'class':'recommend'})
    
    for e in recommend:
        tmpSoup = BeautifulSoup(str(e), "lxml")
        tmpRec = tmpSoup.findAll('li', attrs = {'class':'recommend-answer'})
        if len(tmpRec) == 6:
            total = total + 1
            tmp0 = str(tmpRec[0])
            html0 = BeautifulSoup(tmp0, "lxml")
            current0 = html0.findAll('img', alt = True)
            value = value + int(current0[0].get('alt')[:1])
            
            tmp1 = str(tmpRec[1])
            html1 = BeautifulSoup(tmp1, "lxml")
            current1 = html1.findAll('img', alt = True)
            location = location + int(current1[0].get('alt')[:1])
            
            tmp2 = str(tmpRec[2])
            html2 = BeautifulSoup(tmp2, "lxml")
            current2 = html2.findAll('img', alt = True)
            s_quality = s_quality + int(current2[0].get('alt')[:1])
            
            tmp3 = str(tmpRec[3])
            html3 = BeautifulSoup(tmp3, "lxml")
            current3 = html3.findAll('img', alt = True)
            rooms = rooms + int(current3[0].get('alt')[:1])
            
            tmp4 = str(tmpRec[4])
            html4 = BeautifulSoup(tmp4, "lxml")
            current4 = html4.findAll('img', alt = True)
            clean = clean + int(current4[0].get('alt')[:1])
            
            tmp5 = str(tmpRec[5])
            html5 = BeautifulSoup(tmp5, "lxml")
            current5 = html5.findAll('img', alt = True)
            service = service + int(current5[0].get('alt')[:1])
    url_link = link
    html_page = urllib2.urlopen(url_link).read()
    bs = BeautifulSoup(html_page, "lxml")
    count = 0
    next_page = bs.findAll('a', attrs = {'class':'nav next rndBtn ui_button primary taLnk'})
    
    template = url_link[:-16] + 'or'
    print template
    html_page = urllib2.urlopen(url_link).read()
    bs = BeautifulSoup(html_page, "lxml")
    count = 0
    next_div = bs.findAll('a', attrs = {'class':'nav next rndBtn ui_button primary taLnk'})
    current = int(next_div[0].get('href')[-1:])
    while count < 10:
        url = tmplate + str(current)
        print url
        driver.get(url)
    
        page_source = driver.page_source
        soup_2 = BeautifulSoup(page_source, "lxml")
        rating = soup_2.findAll('ul', attrs = {'class':'recommend'})
        for r in rating:
            ratingSoup = BeautifulSoup(str(r), "lxml")
            ratingRec = ratingSoup.findAll('li', attrs = {'class':'recommend-answer'})
            print len(ratingRec)
            if len(ratingRec) == 6:
                total = total + 1
        current = current + 6
        count = count + 1

https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or
https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or9
3
6
6
6
6
https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or15
3
6
6
6
6
https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or21
3
6
3
https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or27
3
6
3
3
6
3
6
https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or33
3
3
6
3
6
6
https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or39
3
3
3
6
6
https://www.tripadvisor.com/ShowUserReviews-g60745-d89615-r356857589-Taj_Boston-Boston_Massachusetts.html#or45
3
6
6
3
6
3
6
https://www.tripadvisor.com/ShowUserRev

KeyboardInterrupt: 

In [43]:
import urllib2
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url_link = 'https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#CHECK_RATES_CONT'
html_page = urllib2.urlopen(url_link).read()
bs = BeautifulSoup(html_page, "lxml")
c = 0
my_next = bs.findAll('a', attrs = {'class':'nav next rndBtn ui_button primary taLnk'})
driver = webdriver.Firefox()

tmplate = 'https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or'
current = int(my_next[0].get('href')[-1:])
while c < 20:
    url = tmplate + str(current)
    print url
    driver.get(url)
    
    element = driver.page_source
    soup_2 = BeautifulSoup(element, "lxml")
    rating = soup_2.findAll('ul', attrs = {'class':'recommend'})
    for e in rating:
        ratingSoup = BeautifulSoup(str(e), "lxml")
        ratingRec = ratingSoup.findAll('li', attrs = {'class':'recommend-answer'})
        print len(ratingRec)
        if len(ratingRec) == 6:
            tmp0 = str(ratingRec[0])
            html0 = BeautifulSoup(tmp0, "lxml")
            current0 = html0.findAll('img', alt = True)
            current0[0].get('alt')
            
            tmp1 = str(ratingRec[1])
            html1 = BeautifulSoup(tmp1, "lxml")
            current1 = html1.findAll('img', alt = True)
            current1[0].get('alt')
            
            tmp2 = str(ratingRec[2])
            html2 = BeautifulSoup(tmp2, "lxml")
            current2 = html2.findAll('img', alt = True)
            current2[0].get('alt')
            
            tmp3 = str(ratingRec[3])
            html3 = BeautifulSoup(tmp3, "lxml")
            current3 = html3.findAll('img', alt = True)
            current3[0].get('alt')
            
            tmp4 = str(ratingRec[4])
            html4 = BeautifulSoup(tmp4, "lxml")
            current4 = html4.findAll('img', alt = True)
            current4[0].get('alt')
            
            tmp5 = str(ratingRec[5])
            html5 = BeautifulSoup(tmp5, "lxml")
            current5 = html5.findAll('img', alt = True)
            current5[0].get('alt')
    current = current + 6
    c = c + 1
    print '\n'


https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or7
3
3
3
3
3


https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or13
3
3
3
3
3


https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or19
3
3
3


https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or25
3
3
3
3
3


https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or31
3
3
3


https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or37
3
3
3
3
3


https://www.tripadvisor.com/ShowUserReviews-g60745-d222955-r355815318-The_Ritz_Carlton_Boston-Boston_Massachusetts.html#or43
3
3
3
3
3
3


https://www.tripadvisor.com/ShowUserReviews-g607

** Task 2 (20 pts) **

Now, we will use regression to analyze this information. First, we will fit a linear regression model that predicts the average rating. For example, for the hotel above, the average rating is

$$ \text{AVG_SCORE} = \frac{1*31 + 2*33 + 3*98 + 4*504 + 5*1861}{2527}$$

Use the model to analyze the important factors that decide the $\text{AVG_SCORE}$.

** Task 3 (30 pts) **

Finally, we will use logistic regression to decide if a hotel is _excellent_ or not. We classify a hotel as _excellent_ if more than **60%** of its ratings are 5 stars. This is a binary attribute on which we can fit a logistic regression model. As before, use the model to analyze the data.

-------

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()